## Import

In [30]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox


In [31]:

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
sample_num = 0
captured_num = 0

# loop through frames
while webcam.isOpened():

    # read frame from webcam
    status, frame = webcam.read()
    sample_num = sample_num + 1

    if not status :
        break
    face, confidence = cv.detect_face(frame)
    print(face)
    print(confidence)
    
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        if sample_num % 8 == 0 :
            captured_num = captured_num + 1
            face_in_img = frame[startY:endY, startX:endX, :]
            cv2.imwrite(os.path.join('face','7777',"face1" + str(captured_num) + ".jpg"), face_in_img)
            # cv2.imwrite("face_" + str(captured_num) + ".jpg", face_in_img)

    # display output
    cv2.imshow("captured frames", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()

[]
[]
[[786, 279, 1152, 767]]
[0.9762864]
[[785, 287, 1150, 766]]
[0.9663794]
[[786, 291, 1147, 761]]
[0.94595844]
[[783, 291, 1147, 763]]
[0.9561996]
[[782, 284, 1146, 761]]
[0.967318]
[[781, 282, 1144, 763]]
[0.9705175]
[[780, 276, 1143, 762]]
[0.97493273]
[[779, 279, 1142, 763]]
[0.9738679]
[[779, 275, 1142, 763]]
[0.9771568]
[[777, 275, 1141, 763]]
[0.97902757]
[[778, 275, 1142, 763]]
[0.97888535]
[[778, 273, 1141, 764]]
[0.98049015]
[[780, 266, 1144, 764]]
[0.9862017]
[[779, 267, 1145, 762]]
[0.98648703]
[[779, 264, 1148, 762]]
[0.98750764]
[[779, 262, 1151, 761]]
[0.9885274]
[[779, 264, 1152, 761]]
[0.988493]
[[778, 264, 1154, 763]]
[0.9909343]
[[778, 264, 1156, 763]]
[0.99197257]
[[777, 263, 1157, 761]]
[0.99230075]
[[777, 262, 1158, 760]]
[0.99202126]
[[778, 261, 1159, 761]]
[0.99156696]
[[780, 261, 1160, 761]]
[0.9915929]
[[779, 262, 1162, 762]]
[0.99119407]
[[781, 261, 1162, 762]]
[0.9900316]
[[780, 261, 1163, 764]]
[0.989604]
[[782, 263, 1164, 764]]
[0.9886578]
[[783, 264, 1

# Device & Path

In [32]:
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
torch.backends.mps.is_available()

True

In [33]:
# PATH
DATA_PATH  = '../DATA'

## Hyperparameter Setting

In [35]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [36]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [37]:
# import os
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()
# all_folder_list = os.listdir(TRAIN_PATH)
# label = list(all_folder_list)
# label = [x for x in label if not x.startswith('.')]
# label = le.fit_transform(label)
# label
# # Rename folders with encoded labels
# # for k, v in zip(all_folder_list, label):
# #     os.rename(os.path.join(TRAIN_PATH, k), os.path.join(TRAIN_PATH, str(v)))


In [38]:
os.getcwd()
os.chdir(DATA_PATH)

In [39]:
all_img_list = glob.glob('/face/*/*')
import os

# 데이터셋 디렉토리 경로
dataset_dir = "face"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    if not item.startswith('.'):
        item_path = os.path.join(dataset_dir, item)
        for file in os.listdir(item_path):
            if not file.startswith('.'):
                all_img_list.append(os.path.join(item_path, file))
                folder_list.append(item)

In [40]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : int(str(x).split('/')[1]))
df

,img_path,label
0,face/1069/2.jpg,1069
1,face/1069/1.jpg,1069
2,face/1069/0.jpg,1069
3,face/797/2.jpg,797
4,face/797/1.jpg,797
...,...,...
8297,face/986/6.jpg,986
8298,face/986/2.jpg,986
8299,face/986/3.jpg,986
8300,face/986/1.jpg,986


In [41]:
len(df)

8302

In [43]:
for i in range(len(df['label'])):
    if not df['label'][i] == 7777:
        df['label'][i] = int(0)
df[df['label']==7777]

,img_path,label
2964,face/7777/face11.jpg,7777
2965,face/7777/face172.jpg,7777
2966,face/7777/face166.jpg,7777
2967,face/7777/face167.jpg,7777
2968,face/7777/face173.jpg,7777
...,...,...
3057,face/7777/face156.jpg,7777
3058,face/7777/face142.jpg,7777
3059,face/7777/face143.jpg,7777
3060,face/7777/face157.jpg,7777


In [44]:
df

,img_path,label
0,face/1069/2.jpg,0
1,face/1069/1.jpg,0
2,face/1069/0.jpg,0
3,face/797/2.jpg,0
4,face/797/1.jpg,0
...,...,...
8297,face/986/6.jpg,0
8298,face/986/2.jpg,0
8299,face/986/3.jpg,0
8300,face/986/1.jpg,0


In [45]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [46]:
# le = preprocessing.LabelEncoder()
# train['label'] = le.fit_transform(train['label'])
# val['label'] = le.transform(val['label'])
# train['label'] = le.inverse_transform(train['label'])
# val['label'] = le.inverse_transform(val['label'])
# train

## CustomDataset

In [47]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [48]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [49]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [50]:
# !pip install --upgrade efficientnet_pytorch

In [80]:
# !pip install facenet-pytorch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.8 MB/s eta 0:00:00a 0:00:01


In [101]:
!pip install torch_vggface

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement torch_vggface (from versions: none)
ERROR: No matching distribution found for torch_vggface


In [102]:
from facenet_pytorch import InceptionResnetV1

class BaseModel(nn.Module):
    def __init__(self, num_classes=len(df['label'].unique())):
        super(BaseModel, self).__init__()
        self.backbone = InceptionResnetV1(pretrained='vggface2')
        self.classifier = nn.Linear(512, num_classes)  # Update the input size for the classifier
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


## Train

In [92]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            # Create a dictionary to map each unique label to a numerical value
            # label_map = {label: i for i, label in enumerate(set(labels))}
            # Convert the string labels to numerical labels using the label map
            # numerical_labels = [label_map[label] for label in labels]
            # Create a PyTorch tensor from the numerical labels
            # labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            torch.save(best_model.state_dict(), f'best_model_{epoch}.pth')
    
    return best_model

In [ ]:
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            # Create a dictionary to map each unique label to a numerical value
            # label_map = {label: i for i, label in enumerate(set(labels))}
            # Convert the string labels to numerical labels using the label map
            # numerical_labels = [label_map[label] for label in labels]
            # Create a PyTorch tensor from the numerical labels
            # labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            torch.save(best_model.state_dict(), f'best_model_{epoch}.pth')
    
    return best_model

In [93]:
df

,img_path,label
0,face/1069/2.jpg,0
1,face/1069/1.jpg,0
2,face/1069/0.jpg,0
3,face/797/2.jpg,0
4,face/797/1.jpg,0
...,...,...
8297,face/986/6.jpg,0
8298,face/986/2.jpg,0
8299,face/986/3.jpg,0
8300,face/986/1.jpg,0


In [94]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            # Create a dictionary to map each unique label to a numerical value
            # label_map = {label: i for i, label in enumerate(set(labels))}
            # Convert the string labels to numerical labels using the label map
            # numerical_labels = [label_map[label] for label in labels]
            # Create a PyTorch tensor from the numerical labels
            # labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [96]:
next(iter(train_loader))[0]

tensor([[[[-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          ...,
          [-1.3473, -1.1760, -1.1760,  ..., -2.1008, -2.1008, -2.1008],
          [-1.2959, -1.0904, -1.0733,  ..., -2.1008, -2.1008, -2.1008],
          [-1.1589, -0.9192, -0.9020,  ..., -2.1008, -2.1008, -2.1008]],

         [[-1.9482, -1.9482, -1.9482,  ..., -2.0182, -2.0182, -2.0182],
          [-1.9482, -1.9482, -1.9482,  ..., -2.0182, -2.0182, -2.0182],
          [-1.9307, -1.9307, -1.9307,  ..., -2.0182, -2.0182, -2.0182],
          ...,
          [-0.4601, -0.2850, -0.2850,  ..., -2.0182, -2.0182, -2.0182],
          [-0.4076, -0.1975, -0.1800,  ..., -2.0182, -2.0182, -2.0182],
          [-0.2675, -0.0224, -0.0049,  ..., -2.0182, -2.0182, -2.0182]],

         [[-1.6302, -1.6302, -1.6302,  ..., -1.7870, -1.7870, -1.7870],
          [-1.6302, -1.6302, -

In [103]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(train_loader):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss: [{_train_loss:.5f}], Val Loss: [{_val_loss:.5f}], Val Weighted F1 Score: [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            torch.save(best_model.state_dict(), f'best_model_{epoch}.pth')
    
    return best_model

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(val_loader):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

model = BaseModel(num_classes=len(set(df['label'])))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)


  0%|          | 0/208 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch [1], Train Loss: [0.27677], Val Loss: [0.33724], Val Weighted F1 Score: [0.92061]


  0%|          | 0/208 [00:00<?, ?it/s]

## Inference

In [90]:
from PIL import ImageFont, ImageDraw, Image

# load best model
model.load_state_dict(torch.load('best_model_5.pth'))
model.eval()

# open webcam
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

# loop through frames
while webcam.isOpened():

    # read frame from webcam
    status, frame = webcam.read()
    if not status:
        print('Could not read frame')
        exit()
    
    # apply face detection
    face, confidence = cv.detect_face(frame)

    # loop through detected faces
    for idx, f in enumerate(face):

        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:

            face_region = frame[startY:endY, startX:endX]

            face_region1 = cv2.resize(face_region, (224, 224), interpolation=cv2.INTER_AREA)

            x = np.array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = np.transpose(x, (0, 3, 1, 2))
            x = torch.from_numpy(x).float()
            x = x.to(device)

            with torch.no_grad():
                prediction = model(x)
                _, predicted_class = torch.max(prediction, 1)
                predicted_class = predicted_class.item()
            print(predicted_class)

            if predicted_class == int(7777): 
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.putText(frame, 'Sungjin', (startX, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.putText(frame, 'Another', (startX, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                
    # display output
    cv2.imshow("Real-time face detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()


RuntimeError: Error(s) in loading state_dict for BaseModel:
	Missing key(s) in state_dict: "backbone.conv2d_1a.conv.weight", "backbone.conv2d_1a.bn.weight", "backbone.conv2d_1a.bn.bias", "backbone.conv2d_1a.bn.running_mean", "backbone.conv2d_1a.bn.running_var", "backbone.conv2d_2a.conv.weight", "backbone.conv2d_2a.bn.weight", "backbone.conv2d_2a.bn.bias", "backbone.conv2d_2a.bn.running_mean", "backbone.conv2d_2a.bn.running_var", "backbone.conv2d_2b.conv.weight", "backbone.conv2d_2b.bn.weight", "backbone.conv2d_2b.bn.bias", "backbone.conv2d_2b.bn.running_mean", "backbone.conv2d_2b.bn.running_var", "backbone.conv2d_3b.conv.weight", "backbone.conv2d_3b.bn.weight", "backbone.conv2d_3b.bn.bias", "backbone.conv2d_3b.bn.running_mean", "backbone.conv2d_3b.bn.running_var", "backbone.conv2d_4a.conv.weight", "backbone.conv2d_4a.bn.weight", "backbone.conv2d_4a.bn.bias", "backbone.conv2d_4a.bn.running_mean", "backbone.conv2d_4a.bn.running_var", "backbone.conv2d_4b.conv.weight", "backbone.conv2d_4b.